# Splay木(平衡二分探索木)
## 概要
木が平衡に保たれるため、高速にアクセスできる二分探索木
### 計算量
追加、削除、探索: $O(logN)$

## 参考
- [【木マスター養成講座】7-1. Splay木ってなに〜？説明編【競プロかつっぱ】](https://www.youtube.com/watch?v=M6LcINhgXeM)
- [【木マスター養成講座】7-2. Splay木ってなに〜？実装編１【競プロかつっぱ】](https://www.youtube.com/watch?v=M6LcINhgXeM&t=0s)
- https://onlinejudge.u-aizu.ac.jp/solutions/problem/ITP2_1_A/review/4078592/catupper/C++14

## 方針

### クラス設計
Nodeに対応するclassを作る
```
class Node:
    val
    |- left, right, parent: 親と子のポインタ
    |- size: 部分木のサイズ
    |_ value: 値
    func
    |- rotate(): 回転
    |- splay(): meを上に上げる
    |_ update(): サイズの更新
```

### 回転

```
   pp-> o
        |
    p-> o
       / \
 me-> o   B
     / \
    A   C

       ↓

  pp-> o
       |
  me-> o
      / \
     A   o <-p
        / \
       C   B

```
- 全部で6つの代入
1. `pp.left = pp.right = me` ## ppがいるか確認
1. `me.parent = pp`
1. `me.right = p`
1. `p.parent = me`
1. `p.left = C`
1. `C.parent = p` ## Cがいるか確認

In [4]:
import sys
def err(*args, **kwargs): print(*args, **kwargs, file=sys.stderr)

In [11]:
class SplayNode:
    """スプレー木のノード
    Attributes:
        left, right, parent (SplayNode): 親と子のノード
        size (int): 部分木のサイズ
        value: 値
    """
    def __init__(self, value):
        self.left = self.right = None
        self.parent = None
        self.size = 1
        self.value = value
    
    def rotate(self):
        p = self.parent
        pp = p.parent

        if p.left == self:
            c = self.right
            self.right = p
            p.left = c
        else:
            c = self.left
            self.left = p
            p.right = c

        if pp and pp.left == p:
            pp.left = self
        if pp and pp.right == p:
            pp.right = self
        
        self.parent = pp
        p.parent = self
        if c:
            c.parent = p
        
        # 下から順にupdate
        p.update()
        self.update()

    def state(self):
        if not self.parent:
            return 0
        if self.parent.left == self:
            return 1
        if self.parent.right == self:
            return -1
        return 0

    def update(self):
        self.size = 1
        if self.left:
            self.size += self.left.size
        if self.right:
            self.size += self.right.size

    def splay(self):
        while self.state() != 0:
            if self.parent.state() == 0:
                self.rotate()
            elif self.state() == self.parent.state():
                self.parent.rotate()
                self.rotate()
            else:
                self.rotate()
                self.rotate()


def get(ind, root):
    while True:
        lsize = root.left.size if root.left else 0
        if ind < lsize:
            root = root.left
        if ind == lsize:
            root.splay()
            return root
        if ind > lsize:
            root = root.right
            ind = ind - lsize - 1
    

In [12]:
# Q = int(input())
# queries = [tuple(map(int, input().split())) for _ in range(Q)]

Q = 8
queries = [
    (0, 1,),
    (0, 2,),
    (0, 3,),
    (2,),
    (0, 4,),
    (1, 0,),
    (1, 1,),
    (1, 2,),
]

In [14]:
# AOJ
# https://judge.u-aizu.ac.jp/onlinejudge/description.jsp?id=ITP2_1_A&lang=ja

vecsize = 0

# ノード作成
node = [SplayNode(None) for _ in range(22000)]

# ノードをつなげる
for i in range(Q):
    node[i].parent = node[i+1]
    node[i+1].left = node[i]
    node[i+1].update()

root = node[Q]

for q in queries:
    err(*q)
    if q[0] == 0:
        root = get(vecsize, root)
        vecsize += 1
        root.value = q[1]
    elif q[0] == 1:
        root = get(q[1], root)
        print(root.value)
    else:
        vecsize -= 1

1
2
4


0 1
0 2
0 3
2
0 4
1 0
1 1
1 2


### Listで再実装

In [15]:
def delay(arr, i):
    yield arr[i]

In [35]:
l = [[None, None] for _ in range(10)]
l[1][0] = l[0]
l[2][0] = l[1]
l[3][0] = l[2]
l[4][0] = l[3]

l

[[None, None],
 [[None, None], None],
 [[[None, None], None], None],
 [[[[None, None], None], None], None],
 [[[[[None, None], None], None], None], None],
 [None, None],
 [None, None],
 [None, None],
 [None, None],
 [None, None]]

In [36]:
l = [[None, None] for _ in range(10)]

l[1][0] = l[0]
l[2][0] = l[1]
l[3][0] = l[2]
l[4][0] = l[3]

l

## リベンジ
参考
- http://www.nct9.ne.jp/m_hiroi/light/pyalgo20.html